In [1]:
import pandas 
import tensorflow  as tf
import matplotlib.pyplot as plt 
import numpy as np

In [13]:
paths=r"C:\Users\godof\OneDrive\Desktop\summer\pose detection\3.tflite"
interpreter=tf.lite.Interpreter(model_path=paths)
interpreter.allocate_tensors()

In [3]:
def drawkeypoint(frame,keypoints,confidence_thresh):
    y,x,c=frame.shape
    shaped=np.squeeze(np.multiply(keypoints,[y,x,1]))
    for i in shaped:
        y_cordinate,x_cordinate,conf= i
        if conf>confidence_thresh:
            cv2.circle(frame,(int(x_cordinate),int(y_cordinate)),4,(0,255,0),-3)
                #4,(0,255,0),-1 tells how big the circle is and which colour

In [4]:
def drawedges(frame,keypoints,edges,confidence_thresh):
    y,x,c=frame.shape
    shaped=np.squeeze(np.multiply(keypoints,[y,x,1]))

    for edge,color in edges.items():
        p1,p2=edge
        y1,x1,c1=shaped[p1]
        y2,x2,c2=shaped[p2]
        if (c1>confidence_thresh) &(c2>confidence_thresh):
                cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
import cv2
cap=cv2.VideoCapture("novak.mp4")
while True:
    ret,frame=cap.read()
    
    img=frame.copy()
    img=tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
    img=tf.cast(img,dtype=np.float32)#np.squeeze is a function in NumPy that removes single-dimensional entries from the shape of an array. 

    input=interpreter.get_input_details()
    output=interpreter.get_output_details()

    interpreter.set_tensor(input[0]['index'],np.array(img))
    interpreter.invoke()
    keypints_with_score=interpreter.get_tensor(output[0]['index'])

    
    drawkeypoint(frame,keypints_with_score,0.4)
    drawedges(frame,keypints_with_score,EDGES,0.4)
    
    
    cv2.imshow('Web Cam',frame)
    if cv2.waitKey(1)& 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()